In [1]:
import pandas as pd
import requests
import json

class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        # API Key value from link
        api_key = 'GF0IU3N4VNWYWAV5'
        
        # construct the API URL
        base_url = "https://www.alphavantage.co/query"
        function = "TIME_SERIES_INTRADAY"
        symbol = script
        interval = "1min"
        outputsize = "compact"
        api_url = f"{base_url}?function={function}&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}"
        
        # send the request and parse the response
        response = requests.get(api_url)
        if response.status_code != 200:
            raise Exception("Failed to fetch data. Check the API key and script name.")
        data = response.json()
        self.data[script] = data
    
    def convert_intraday_data(self, script):
        # get the data for the given script
        data = self.data[script]
        
        # extract the time series data
        time_series_data = data["Time Series (1min)"]
        
        # create a list of tuples containing the timestamp and data for each time point
        data_list = [(timestamp, data) for timestamp, data in time_series_data.items()]
        
        # sort the data by timestamp
        data_list.sort(key=lambda x: x[0])
        
        # extract the data into separate lists
        timestamps, data = zip(*data_list)
        opens, highs, lows, closes, volumes = [], [], [], [], []
        for d in data:
            opens.append(float(d["1. open"]))
            highs.append(float(d["2. high"]))
            lows.append(float(d["3. low"]))
            closes.append(float(d["4. close"]))
            volumes.append(int(d["5. volume"]))
        
        # create a pandas DataFrame from the lists
        df = pd.DataFrame({"timestamp": timestamps, "open": opens, "high": highs, "low": lows, "close": closes, "volume": volumes})
        
        # set the timestamp column as the index of the DataFrame
        df.set_index("timestamp", inplace=True)
        
        # convert the timestamp strings to pandas Timestamps
        df.index = pd.to_datetime(df.index)
        
        # store the DataFrame in the data dictionary
        self.data[script] = df
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __contains__(self, key):
        return key in self.data


In [2]:
# create an instance of the ScriptData class
script_data = ScriptData()

# fetch intraday data for the 'GOOGL' script using your Alpha Vantage API key
script_data.fetch_intraday_data('GOOGL')

# convert the fetched data into a pandas DataFrame
script_data.convert_intraday_data('GOOGL')

# access the DataFrame for the 'GOOGL' script
df = script_data['GOOGL']




In [3]:
df

,open,high,low,close,volume
timestamp,,,,,
2023-01-04 17:01:00,88.0388,88.0388,88.0388,88.0388,210
2023-01-04 17:03:00,88.0375,88.0375,88.0375,88.0375,126
2023-01-04 17:08:00,87.9700,87.9700,87.9700,87.9700,1916
2023-01-04 17:09:00,88.0400,88.0700,88.0300,88.0700,1415
2023-01-04 17:10:00,88.0800,88.0800,88.0800,88.0800,828
...,...,...,...,...,...
2023-01-04 19:55:00,87.8800,87.8800,87.8800,87.8800,1052
2023-01-04 19:56:00,87.8700,87.8800,87.8100,87.8100,406
2023-01-04 19:58:00,87.8700,87.8800,87.8700,87.8800,1037


In [4]:

script_data.fetch_intraday_data('AAPL')

script_data.convert_intraday_data('AAPL')

df = script_data['AAPL']





In [5]:
df

,open,high,low,close,volume
timestamp,,,,,
2023-01-04 18:10:00,127.2000,127.2000,127.20,127.2000,601
2023-01-04 18:11:00,127.2400,127.2500,127.23,127.2500,941
2023-01-04 18:12:00,127.2500,127.2500,127.20,127.2000,3349
2023-01-04 18:13:00,127.1801,127.1801,127.15,127.1500,1656
2023-01-04 18:14:00,127.1600,127.2799,127.16,127.2799,6971
...,...,...,...,...,...
2023-01-04 19:56:00,126.7000,126.7000,126.66,126.6600,1136
2023-01-04 19:57:00,126.6600,126.6700,126.66,126.6700,1430
2023-01-04 19:58:00,126.7200,126.7200,126.67,126.7200,2424


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    # create a new column in the DataFrame with the moving average of the close price
    df["indicator"] = df["close"].rolling(timeperiod).mean()
    
    # create a new DataFrame with the timestamp and indicator columns
    result = pd.DataFrame({"timestamp": df.index, "indicator": df["indicator"]})

    return result


In [10]:
# fetch and convert intraday data for the 'GOOGL' script
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

# compute the indicator1 for the 'GOOGL' script with a timeperiod of 5
indicator1_df1 = indicator1(script_data['GOOGL'], timeperiod = 5)


In [11]:
indicator1_df1

,timestamp,indicator
timestamp,,
2023-01-04 17:01:00,2023-01-04 17:01:00,NaN
2023-01-04 17:03:00,2023-01-04 17:03:00,NaN
2023-01-04 17:08:00,2023-01-04 17:08:00,NaN
2023-01-04 17:09:00,2023-01-04 17:09:00,NaN
2023-01-04 17:10:00,2023-01-04 17:10:00,88.03926
...,...,...
2023-01-04 19:55:00,2023-01-04 19:55:00,87.81600
2023-01-04 19:56:00,2023-01-04 19:56:00,87.81800
2023-01-04 19:58:00,2023-01-04 19:58:00,87.84000


In [12]:
script_data = ScriptData()
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

indicator1_df2 = indicator1(script_data['AAPL'], timeperiod = 5)

In [13]:
indicator1_df2

,timestamp,indicator
timestamp,,
2023-01-04 18:10:00,2023-01-04 18:10:00,NaN
2023-01-04 18:11:00,2023-01-04 18:11:00,NaN
2023-01-04 18:12:00,2023-01-04 18:12:00,NaN
2023-01-04 18:13:00,2023-01-04 18:13:00,NaN
2023-01-04 18:14:00,2023-01-04 18:14:00,127.21598
...,...,...
2023-01-04 19:56:00,2023-01-04 19:56:00,126.67000
2023-01-04 19:57:00,2023-01-04 19:57:00,126.67400
2023-01-04 19:58:00,2023-01-04 19:58:00,126.69000


In [14]:

class Strategy:
    def __init__(self, script):
        self.script_data = ScriptData()
    
    def fetch_intraday_data(self, script):
        self.script_data.fetch_intraday_data(script)
        self.script_data.convert_intraday_data(script)
        self.df = self.script_data[script]
    
    def compute_indicator_data(self):
        self.indicator_df = indicator1(self.df, 5)
    
    def get_signals_data(self):
        # create a new column in the DataFrame with the moving average of the close price
        self.df["indicator"] = self.indicator_df["indicator"]
        
        # create a new column in the DataFrame with the signals
        self.df["signal"] = "NO_SIGNAL"
        self.df.loc[self.df["indicator"] > self.df["close"], "signal"] = "BUY"
        self.df.loc[self.df["indicator"] < self.df["close"], "signal"] = "SELL"
        
        # create a new DataFrame with the timestamp and signal columns
        self.initial_signals = pd.DataFrame({"timestamp": self.df.index, "signal": self.df["signal"]})
        
        # filter the signals DataFrame to include only the rows with a 'BUY' or 'SELL' signal
        self.signals = strategy.initial_signals[strategy.initial_signals["signal"].isin(["BUY", "SELL"])]
        
        # track the previous signal
        prev_signal = self.signals.iloc[0]["signal"]
        
        # create a new DataFrame to store the timestamps and signals
        self.signals_df = pd.DataFrame(columns=["timestamp", "signal"])
        
        # iterate through the signals DataFrame and add the timestamp and signal to the signals_df DataFrame if the signal is different from the previous signal
        for i, row in self.signals.iterrows():
            if row["signal"] != prev_signal:
                # update the signals_df DataFrame by concatenating the current row
                self.signals_df = pd.concat([self.signals_df, pd.DataFrame({"timestamp": [row["timestamp"]], "signal": [row["signal"]]})])
            prev_signal = row["signal"]
        
        return self.signals_df

        


In [16]:
# create an instance of the Strategy class
strategy = Strategy('NVDA')

# fetch intraday data for the script 'NVDA'
strategy.fetch_intraday_data('NVDA')

# compute the indicator data
strategy.compute_indicator_data()

# get the signals data
signals_df = strategy.get_signals_data()


In [17]:
signals_df

,timestamp,signal
0,2023-01-04 16:44:00,SELL
0,2023-01-04 16:54:00,BUY
0,2023-01-04 16:59:00,SELL
0,2023-01-04 17:04:00,BUY
0,2023-01-04 17:08:00,SELL
0,2023-01-04 17:17:00,BUY
0,2023-01-04 17:19:00,SELL
0,2023-01-04 17:33:00,BUY
0,2023-01-04 17:39:00,SELL
0,2023-01-04 17:49:00,BUY
